In [21]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import (classification_report, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,
                            silhouette_score)
import joblib


In [2]:
# Jaime

df_modelo = pd.read_csv("/Users/jaime/Documents/ICAI/Quinto/Desarrollo Apps de Visualización/Trabajo/DataFinal/datos_modelo.csv", index_col=0)

In [24]:
df_modelo.head()

,host_is_superhost,neighbourhood_group_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,price,minimum_nights,availability_365,number_of_reviews,calculated_host_listings_count,puntuacion
0,True,Chamartín,40.45724,-3.67688,Private room,2,1.0,1.0,60.0,1,180,78,1,4.583333
1,False,Latina,40.40381,-3.74130,Private room,1,1.0,1.0,31.0,4,364,33,2,4.653846
4,False,Arganzuela,40.38975,-3.69018,Private room,1,1.0,1.0,26.0,2,365,149,1,4.612613
6,False,Centro,40.41476,-3.70418,Entire home/apt,2,1.0,1.0,65.0,5,351,170,3,4.596774
7,False,Centro,40.41259,-3.70105,Entire home/apt,2,1.0,1.0,54.0,3,0,8,1,4.571429


In [3]:
df_modelo['neighbourhood_group_cleansed'] = df_modelo['neighbourhood_group_cleansed'].astype('category')
df_modelo['room_type'] = df_modelo['room_type'].astype('category')

In [4]:
df_modelo = pd.get_dummies(df_modelo)

# Split Training y Test

In [5]:
# Dividir train-test
X = df_modelo.drop(['price'], axis = 1)
y = df_modelo['price']
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size = 0.2, random_state = 123)

# Modelo RandomForest

In [6]:
model = RandomForestRegressor()
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Según la documentación de SKLearn, los más importantes: max_features, max_depth. A continuación, con RandomSearchCV, se probará el modelo variando los hiperparametros, para obtener el mejor resultado

In [7]:
n_estimators = [int(x) for x in np.linspace(start = 150, stop = 800, num = 20)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(10, 300, num = 20)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]

In [8]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [32]:
rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 20, cv = 10, random_state=123)
rf_random.fit(X_train, y_train)

In [54]:
rf_random.best_params_

{'n_estimators': 697,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 300,
 'bootstrap': False}

In [15]:
rf_best = RandomForestRegressor(n_estimators=697, max_features= 'log2', max_depth = 300, min_samples_split=2, min_samples_leaf= 2, bootstrap=False)
rf_best.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=300, max_features='log2',
                      min_samples_leaf=2, n_estimators=697)

In [19]:
predictions = rf_best.predict(X_test)

In [56]:
#Mean Absolute Error 
print('MAE:', metrics.mean_absolute_error(y_test, predictions))

#Mean Squared Error
print('MSE:', metrics.mean_squared_error(y_test, predictions))

#Root Mean Squared Error
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

#R2
print('R2:', metrics.r2_score(y_test, predictions))

MAE: 60.98937162384864
MSE: 23121.095369118288
RMSE: 152.05622436821943
R2: 0.41600908780110735


In [22]:
joblib_file = "modelo_RF.pkl"  
joblib.dump(rf_best, joblib_file)

['modelo_RF.pkl']